In [1]:
import sys, os

ms_udfix = lambda x:x.replace('\\', '/')[:-1]

sys.path.append("samples/coco/")
from coco import *

C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\pycocotools\coco.py:49: UserWarning: matplotlib.pyplot as already been imported, this call will have no effect.
  import matplotlib; matplotlib.use('Agg')
Using TensorFlow backend.


In [2]:
train_ud = ms_udfix(r"C:\Users\Public\Segmentation\seg-countertops\annotationsWithImages\\")
V_ROOT_DIR = ms_udfix(r"C:\Users\Public\Mask_RCNN-master\\")

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ms_udfix(r"C:\Users\Public\Segmentation\seg-countertops\\"), "logs/")
if not(os.path.exists(MODEL_DIR)):os.mkdir(MODEL_DIR)
#MODEL_DIR = os.path.join(ms_udfix(r"C:\Users\Public\Mask_RCNN-master\\"), "logs")

logs = os.path.join(ms_udfix(r"C:\Users\dominics.CORP\Documents\dev\MaskRCNN_ds\model_logs\\"))
if not(os.path.exists(logs)):
    os.mkdir(logs)

# Path to trained weights file
# Download this file and place in the root of your 
# project (See README file for details)
#You should have trained with this: COCO_MODEL_PATH = "C:\Users\Public\Mask_RCNN-master\mask_rcnn_coco.h5")
#COCO_MODEL_PATH = os.path.join(V_ROOT_DIR, "mask_rcnn_coco1_0006.h5")
COCO_MODEL_PATH = os.path.join(V_ROOT_DIR, "mask_rcnn_coco.h5")

#after 10cycles of Training network heads
#COCO_MODEL_PATH = ms_udfix(r"C:\Users\dominics.CORP\Documents\dev\MaskRCNN_ds\model_logs\coco20181105T1327\mask_rcnn_coco_0010.h5 ")
# Directory of images to run detection on
IMAGE_DIR = train_ud##os.path.join(ROOT_DIR, "graniteimages")

In [3]:
args = """python coco.py train --dataset=C:/Users/Public/Mask_RCNN-master/segmentation/kitchen/ --model=%s --logs %s"""%(COCO_MODEL_PATH, logs)

sys.argv = args.split(' ')[1:]

args = make_parser()
print("Command: ", args.command)
print("Model: ", args.model)
print("Dataset: ", args.dataset)
print("Year: ", args.year)
print("Logs: ", args.logs)
print("Auto Download: ", args.download)

Command:  train
Model:  C:/Users/Public/Mask_RCNN-master/mask_rcnn_coco.h5
Dataset:  C:/Users/Public/Mask_RCNN-master/segmentation/kitchen/
Year:  NONE
Logs:  C:/Users/dominics.CORP/Documents/dev/MaskRCNN_ds/model_logs/
Auto Download:  False


In [5]:
#Load Model for training
config = CocoConfig()
config.IMAGES_PER_GPU = 1
config.BATCH_SIZE=1
config.STEPS_PER_EPOCH=100
config.VALIDATION_STEPS=10
config.NUM_CLASSES = 2
config.LOSS_WEIGHTS ={'rpn_bbox_loss': 1.0, 
                      'mrcnn_mask_loss': 4.0, 
                      'rpn_class_loss': 1.0, 
                      'mrcnn_class_loss': 1.0, 
                      'mrcnn_bbox_loss': 1.0}

model = modellib.MaskRCNN(mode="training", config=config,
                                  model_dir=args.logs)

# Select weights file to load
if args.model.lower() == "coco":
    model_path = COCO_MODEL_PATH
elif args.model.lower() == "last":
    # Find last trained weights
    model_path = model.find_last()[1]
elif args.model.lower() == "imagenet":
    # Start from ImageNet trained weights
    model_path = model.get_imagenet_weights()
else:
    model_path = args.model
print(model_path)
# Load weights
print("Loading weights ", model_path)
print('*************')
model.load_weights(model_path, by_name=True,exclude=["mrcnn_class_logits", "mrcnn_bbox_fc","mrcnn_bbox", "mrcnn_mask"])
print('###############')

config.display()

C:/Users/Public/Mask_RCNN-master/mask_rcnn_coco.h5
Loading weights  C:/Users/Public/Mask_RCNN-master/mask_rcnn_coco.h5
*************
###############

Configurations:
BACKBONE                       resnet101
BACKBONE_SHAPES                [[256 256]
 [128 128]
 [ 64  64]
 [ 32  32]
 [ 16  16]]
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEAR

In [6]:
#Train
dataset_train = CocoDataset()
dataset_train.load_coco(args.dataset, "train")
c = 0;
for img in dataset_train.image_info:c+=len(img['annotations'])
print('dataset_train.image_info: [n=%i] total annotations: %i'%(len(dataset_train.image_info), c))
#COCO API consumes data.json [info,categories,annotations,categories] 
#for details: https://github.com/cocodataset/cocoapi/blob/master/PythonAPI/pycocotools/coco.py

dataset_train.prepare()
#dataset_train.load_coco(args.dataset, "val35k")

#Validation dataset
dataset_val = CocoDataset()
dataset_val.load_coco(args.dataset, "train")
dataset_val.prepare()

# *** This training schedule is an example. Update to your needs ***

#Training - Stage 1
print("Training network heads")
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=10,
            layers='heads')

# Training - Stage 2
#Finetune layers from ResNet stage 4 and up

print("Fine tune Resnet stage 4 and up")
model.train(dataset_train, dataset_val,
             learning_rate=config.LEARNING_RATE,
             epochs=50,
             layers='4+')

# Training - Stage 3
# Fine tune all layers
# print("Fine tune all layers")
# model.train(dataset_train, dataset_val,
#              learning_rate=config.LEARNING_RATE / 10,
#              epochs=10,
#              layers='all')


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
CocoDataset(utils.Dataset): image_dir: C:/Users/Public/Mask_RCNN-master/segmentation/kitchen/newtrain json_path: annotations/data.json
dataset_train.image_info: [n=77] total annotations: 137
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
CocoDataset(utils.Dataset): image_dir: C:/Users/Public/Mask_RCNN-master/segmentation/kitchen/newtrain json_path: annotations/data.json
Training network heads

Starting at epoch 0. LR=0.001

Checkpoint Path: C:/Users/dominics.CORP/Documents/dev/MaskRCNN_ds/model_logs/coco20181107T0005\mask_rcnn_coco_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared      

C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\ops\gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
100/100 [==============================] - 109s 1s/step - loss: 3.3132 - rpn_class_loss: 0.0195 - rpn_bbox_loss: 0.7145 - mrcnn_class_loss: 0.1024 - mrcnn_bbox_loss: 0.7917 - mrcnn_mask_loss: 1.6851 - val_loss: 2.1020 - val_rpn_class_loss: 0.0139 - val_rpn_bbox_loss: 0.3222 - val_mrcnn_class_loss: 0.0530 - val_mrcnn_bbox_loss: 0.4713 - val_mrcnn_mask_loss: 1.2415
Epoch 2/10
100/100 [==============================] - 97s 975ms/step - loss: 2.6225 - rpn_class_loss: 0.0129 - rpn_bbox_loss: 0.5309 - mrcnn_class_loss: 0.0435 - mrcnn_bbox_loss: 0.5323 - mrcnn_mask_loss: 1.5029 - val_loss: 1.8299 - val_rpn_class_loss: 0.0099 - val_rpn_bbox_loss: 0.2369 - val_mrcnn_class_loss: 0.0298 - val_mrcnn_bbox_loss: 0.3806 - val_mrcnn_mask_loss: 1.1728
Epoch 3/10
100/100 [==============================] - 98s 980ms/step - loss: 2.1478 - rpn_class_loss: 0.0117 - rpn_bbox_loss: 0.3568 - mrcnn_class_loss: 0.0391 - mrcnn_bbox_loss: 0.4355 - mrcnn_mask_loss: 1.3047 - val_loss: 2.0316 - val_rpn_cla

C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\ops\gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 11/50
100/100 [==============================] - 132s 1s/step - loss: 1.1863 - rpn_class_loss: 0.0059 - rpn_bbox_loss: 0.2320 - mrcnn_class_loss: 0.0249 - mrcnn_bbox_loss: 0.1982 - mrcnn_mask_loss: 0.7253 - val_loss: 0.9338 - val_rpn_class_loss: 0.0043 - val_rpn_bbox_loss: 0.2745 - val_mrcnn_class_loss: 0.0248 - val_mrcnn_bbox_loss: 0.1517 - val_mrcnn_mask_loss: 0.4785
Epoch 12/50
100/100 [==============================] - 116s 1s/step - loss: 1.2133 - rpn_class_loss: 0.0048 - rpn_bbox_loss: 0.2826 - mrcnn_class_loss: 0.0247 - mrcnn_bbox_loss: 0.2022 - mrcnn_mask_loss: 0.6989 - val_loss: 1.5048 - val_rpn_class_loss: 0.0054 - val_rpn_bbox_loss: 0.6520 - val_mrcnn_class_loss: 0.0255 - val_mrcnn_bbox_loss: 0.1433 - val_mrcnn_mask_loss: 0.6787
Epoch 13/50
100/100 [==============================] - 116s 1s/step - loss: 1.1171 - rpn_class_loss: 0.0058 - rpn_bbox_loss: 0.3145 - mrcnn_class_loss: 0.0276 - mrcnn_bbox_loss: 0.1816 - mrcnn_mask_loss: 0.5878 - val_loss: 0.8990 - val_rpn_clas

Epoch 33/50
100/100 [==============================] - 109s 1s/step - loss: 0.3867 - rpn_class_loss: 0.0022 - rpn_bbox_loss: 0.0700 - mrcnn_class_loss: 0.0083 - mrcnn_bbox_loss: 0.0333 - mrcnn_mask_loss: 0.2729 - val_loss: 0.3332 - val_rpn_class_loss: 0.0019 - val_rpn_bbox_loss: 0.0200 - val_mrcnn_class_loss: 0.0064 - val_mrcnn_bbox_loss: 0.0268 - val_mrcnn_mask_loss: 0.2781
Epoch 34/50
100/100 [==============================] - 110s 1s/step - loss: 0.3511 - rpn_class_loss: 0.0018 - rpn_bbox_loss: 0.0510 - mrcnn_class_loss: 0.0077 - mrcnn_bbox_loss: 0.0284 - mrcnn_mask_loss: 0.2623 - val_loss: 0.3578 - val_rpn_class_loss: 0.0012 - val_rpn_bbox_loss: 0.0601 - val_mrcnn_class_loss: 0.0064 - val_mrcnn_bbox_loss: 0.0175 - val_mrcnn_mask_loss: 0.2726
Epoch 35/50
100/100 [==============================] - 110s 1s/step - loss: 0.3707 - rpn_class_loss: 0.0021 - rpn_bbox_loss: 0.0539 - mrcnn_class_loss: 0.0083 - mrcnn_bbox_loss: 0.0303 - mrcnn_mask_loss: 0.2760 - val_loss: 0.3375 - val_rpn_clas

In [ ]:
import tensorflow as tf
from tensorflow.python.ops.variables import Variable

for k, v in locals().items():
    if type(v) !=str and type(v) !=list: print(k, type(v), v)
    if type(v) is Variable or type(v) is tf.Tensor:
        print("{0}: {1}".format(k, v))

In [ ]:
# Validation dataset

dataset_val = CocoDataset()
coco = dataset_val.load_coco(args.dataset, "minival", return_coco=True)
dataset_val.prepare()
print("Running COCO evaluation on {} images.".format(args.limit))
evaluate_coco(model, dataset_val, coco, "bbox", limit=int(args.limit))